In [ ]:
import os
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from fastervit import create_model
import pandas as pd
from tqdm import tqdm

# Define the number of classes in your custom dataset
num_classes = 10

# Create the model architecture
model = create_model('faster_vit_3_224', pretrained=False)

# Modify the final classification layer
model.head = torch.nn.Linear(model.head.in_features, num_classes)

# Move the model to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

# Load the trained model weights
model.load_state_dict(torch.load('/kaggle/input/mode-4/faster_vit3_normal_final.pth', map_location=device))
model.eval()

# Define data transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = preprocess(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Function to make predictions
def predict(image_path, model):
    image = load_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
    return probabilities.squeeze().tolist()

# List of class names
class_names = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body',
               'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']

# Directory containing test images
test_dir = '/kaggle/input/test-data/Testing set/Images'  # Update this path

# List to store results
results = []

# Process each image in the test directory
for filename in tqdm(os.listdir(test_dir)):
    if filename.endswith('.jpg'):
        image_path = os.path.join(test_dir, filename)
        probabilities = predict(image_path, model)
        
        # Find the predicted class
        predicted_class = class_names[probabilities.index(max(probabilities))]
        
        # Add results to the list
        results.append({
            'image_path': filename,
            **{class_name: prob for class_name, prob in zip(class_names, probabilities)},
            'predicted_class': predicted_class
        })

# Create a DataFrame
df = pd.DataFrame(results)

# Reorder columns to match the desired format
column_order = ['image_path'] + class_names + ['predicted_class']
df = df[column_order]

# Save to Excel
output_file = 'test_prediction_results.xlsx'
df.to_excel(output_file, index=False)

print(f"Results saved to {output_file}")


In [ ]:
import os
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from fastervit import create_model
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score
import json
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, auc, recall_score, f1_score, balanced_accuracy_score
# Define metrics calculation functions
def calculate_specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    return specificity

def generate_metrics_report(y_true, y_pred):
    class_columns = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body', 
                    'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']
    metrics_report = {}
    
    y_true_classes = np.argmax(y_true, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    class_report = classification_report(y_true_classes, y_pred_classes, 
                                      target_names=class_columns, 
                                      output_dict=True, 
                                      zero_division=0)
    
    # Calculate AUC-ROC scores
    auc_roc_scores = {}
    for i, class_name in enumerate(class_columns):
        try:
            auc_roc_scores[class_name] = roc_auc_score(y_true[:, i], y_pred[:, i])
        except ValueError:
            auc_roc_scores[class_name] = 0.0
    
    mean_auc_roc = np.mean(list(auc_roc_scores.values()))
    auc_roc_scores['mean_auc'] = mean_auc_roc
    
    # Calculate specificity scores
    specificity_scores = {}
    for i, class_name in enumerate(class_columns):
        specificity_scores[class_name] = calculate_specificity(y_true[:, i], 
                                                             (y_pred[:, i] >= 0.5).astype(int))
    
    mean_specificity = np.mean(list(specificity_scores.values()))
    specificity_scores['mean_specificity'] = mean_specificity
    
    # Calculate average precision scores
    average_precision_scores = {}
    for i, class_name in enumerate(class_columns):
        try:
            precision, recall, _ = precision_recall_curve(y_true[:, i], y_pred[:, i])
            average_precision_scores[class_name] = auc(recall, precision)
        except ValueError:
            average_precision_scores[class_name] = 0.0
    
    mean_average_precision = np.mean(list(average_precision_scores.values()))
    average_precision_scores['mean_average_precision'] = mean_average_precision
    
    # Calculate sensitivity scores
    sensitivity_scores = {}
    for i, class_name in enumerate(class_columns):
        try:
            sensitivity_scores[class_name] = recall_score(y_true[:, i], 
                                                        (y_pred[:, i] >= 0.5).astype(int), 
                                                        zero_division=0)
        except ValueError:
            sensitivity_scores[class_name] = 0.0
    
    mean_sensitivity = np.mean(list(sensitivity_scores.values()))
    sensitivity_scores['mean_sensitivity'] = mean_sensitivity
    
    # Calculate F1 scores
    f1_scores = {}
    for i, class_name in enumerate(class_columns):
        try:
            f1_scores[class_name] = f1_score(y_true[:, i], 
                                           (y_pred[:, i] >= 0.5).astype(int), 
                                           zero_division=0)
        except ValueError:
            f1_scores[class_name] = 0.0
    
    mean_f1_score = np.mean(list(f1_scores.values()))
    f1_scores['mean_f1_score'] = mean_f1_score
    
    # Calculate balanced accuracy
    balanced_accuracy_scores = balanced_accuracy_score(y_true_classes, y_pred_classes)
    
    # Compile all metrics
    metrics_report.update(class_report)
    metrics_report['auc_roc_scores'] = auc_roc_scores
    metrics_report['specificity_scores'] = specificity_scores
    metrics_report['average_precision_scores'] = average_precision_scores
    metrics_report['sensitivity_scores'] = sensitivity_scores
    metrics_report['f1_scores'] = f1_scores
    metrics_report['mean_auc'] = mean_auc_roc
    metrics_report['mean_specificity'] = mean_specificity
    metrics_report['mean_average_precision'] = mean_average_precision
    metrics_report['mean_sensitivity'] = mean_sensitivity
    metrics_report['mean_f1_score'] = mean_f1_score
    metrics_report['balanced_accuracy'] = balanced_accuracy_scores
    
    metrics_report_json = json.dumps(metrics_report, indent=4)
    return metrics_report_json

# Define the number of classes in your custom dataset
num_classes = 10

# Create the model architecture
model = create_model('faster_vit_3_224', pretrained=False)
# Modify the final classification layer
model.head = torch.nn.Linear(model.head.in_features, num_classes)

# Move the model to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load the trained model weights
model.load_state_dict(torch.load('/kaggle/input/mode-4/faster_vit3_normal_final.pth', map_location=device))
model.eval()

# Define data transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Function to load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = preprocess(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Function to make predictions
def predict(image_path, model, class_names):
    image = load_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        predicted_class = class_names[probabilities.argmax().item()]
    return probabilities.squeeze().tolist(), predicted_class

# List of class names
class_names = ['Angioectasia', 'Bleeding', 'Erosion', 'Erythema', 'Foreign Body',
               'Lymphangiectasia', 'Normal', 'Polyp', 'Ulcer', 'Worms']

# Base directory
base_dir = '/kaggle/input/example/example/Dataset/val'

# Initialize lists to store true labels and predictions
y_true_list = []
y_pred_list = []

# Initialize the DataFrame
data = []

# Recursively traverse the directory structure with progress bar
for root, dirs, files in tqdm(os.walk(base_dir), desc="Processing images"):
    if files:  # Only process if there are files in the directory
        # Get the true class from the directory name
        true_class = os.path.basename(root)
        if true_class in class_names:  # Make sure it's a valid class directory
            for file in files:
                if file.endswith('.jpg'):
                    # Construct the relative image path
                    image_path = os.path.join(root, file).replace(base_dir + '\\', '')
                    
                    # Make the prediction
                    probabilities, predicted_class = predict(os.path.join(root, file), model, class_names)
                    
                    # Create one-hot encoded true label
                    true_label = np.zeros(len(class_names))
                    true_label[class_names.index(true_class)] = 1
                    
                    # Add to lists for metrics calculation
                    y_true_list.append(true_label)
                    y_pred_list.append(probabilities)
                    
                    # Add the row to the data list
                    data.append({
                        'image_path': image_path,
                        'true_class': true_class,
                        **{class_name: prob for class_name, prob in zip(class_names, probabilities)},
                        'predicted_class': predicted_class
                    })

# Convert lists to numpy arrays
y_true = np.array(y_true_list)
y_pred = np.array(y_pred_list)

# Generate metrics report
metrics_report = generate_metrics_report(y_true, y_pred)

# Create the DataFrame and save it to Excel
df = pd.DataFrame(data)
df.to_excel('validation_prediction_results.xlsx', index=False)

# Save metrics report to JSON
with open('metrics_report.json', 'w') as f:
    f.write(metrics_report)

# Print metrics report
print("\nMetrics Report:")
print(metrics_report)

print("\nResults saved to predictions_output.xlsx and metrics_report.json")